This is the second iteration of a notebook examining classification methods for EEG motor imagery events using brain-computer interface devices. It is based on the research from Lun et al. in the following article, as well as other related literature.

https://pmc.ncbi.nlm.nih.gov/articles/PMC7522466/

The basic idea is that subjects wearing brain-computer interface devices (brain-wave reading headsets) imagine themselves closing their left or right fist, but don't actually close them. We create a classifier model to determine when the user does this given streaming EEG data.

The pipeline looks like downloading the dataset, pre-processing the data, training the model, and then loading it to a device for inference via ONNX.

This notebook is a cleaned up version of the other notebook in this directory. All the functions have been rewritten using pytorch and more modern methods, and the python functions can be found in this same repository. This notebook is for experimentation and parameter tuning.

Let's start by loading our required libraries:

In [4]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.

In [47]:
import os
import mne
from mne.datasets import eegbci
from mne.io import read_raw_edf
from mne import concatenate_raws
from urllib.error import HTTPError
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split


Now let's define a function to download the dataset from Physionet. We'll also need to define the subjects and runs we'd like to download, as well as the output directory.  

In [8]:
subjects = [1,2,3,4,5]
runs = [3,4,7,8,11,12]
data_dir = "/content/motor-classification/data"

In [19]:
def download_physionet(subjects, runs, out_dir):
    """
    Download EEG motor imagery data for given subjects and runs using MNE's eegbci API.

    Args:
        subjects (list of int): List of subject numbers to download.
        runs (list of int): List of run numbers to download for each subject.
        out_dir (str): Directory to save downloaded EDF files.

    Returns:
        list: List of file paths to downloaded EDF files.
    """
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    raw_fnames = []
    for subject in subjects:
        try:
            raw_fnames += eegbci.load_data(subject, runs, path=out_dir)
        except HTTPError as err:
            print(f"Error downloading subject {subject}: {err}")
    return raw_fnames

In [20]:
download_physionet(subjects,runs,out_dir=data_dir)

Do you want to set the path:
    /content/motor-classification/data
as the default EEGBCI dataset path in the mne-python config [y]/n? y
Attempting to create new mne-python configuration file:
/root/.mne/mne-python.json
Download complete in 01m30s (14.9 MB)


Download complete in 01m28s (14.6 MB)


Download complete in 01m31s (14.9 MB)


Download complete in 01m35s (14.6 MB)


Download complete in 01m22s (14.6 MB)


[PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R03.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R04.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R07.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R08.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R11.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R12.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S002/S002R03.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S002/S002R04.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S002/S002R07.edf'),
 PosixPath('/content/motor-classification/data/MNE-eegbci-data/f

Next we'll preprocess the data.

In [26]:
raw_dir = "/content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0"
processed_dir = "/content/motor-classification/data/processed"


In [40]:
def preprocess_edf(edf_path, event_id=None, tmin=0, tmax=0.25, l_freq=1.0, h_freq=40.0, montage='standard_1005'):
    """
    Loads and preprocesses a single EDF file using MNE.

    Args:
        edf_path (str): Path to the EDF file.
        event_id (dict, optional): Mapping of event labels to integer codes. Defaults to {'T0': 1, 'T1': 2, 'T2': 3}.
        tmin (float, optional): Start time before event (in seconds). Defaults to 0.
        tmax (float, optional): End time after event (in seconds). Defaults to 0.25.
        l_freq (float, optional): Low cutoff frequency for bandpass filter. Defaults to 1.0.
        h_freq (float, optional): High cutoff frequency for bandpass filter. Defaults to 40.0.
        montage (str, optional): Name of the electrode montage to use. Defaults to 'standard_1005'.

    Returns:
        tuple: (X, y)
            X (np.ndarray): Array of shape (n_epochs, n_channels, n_times) containing the epoched EEG data.
            y (np.ndarray): Array of shape (n_epochs,) containing the event labels for each epoch.
    """
    # Load raw EDF file
    raw = mne.io.read_raw_edf(edf_path, preload=True)

    # Apply bandpass filter
    raw.filter(l_freq, h_freq, fir_design='firwin')

    # Set montage for electrode positions, allowing unknown channel positions
    raw.set_montage(montage, on_missing='warn')

    # Set up event mapping including T0 (resting state)
    if event_id is None:
        event_id = dict(T0=1, T1=2, T2=3)

    # Extract events from annotations
    events, _ = mne.events_from_annotations(raw, event_id=event_id)

    # Select EEG channels, excluding any marked as bad
    picks = mne.pick_types(raw.info, eeg=True, exclude='bads')

    # Create epochs of 0.25 seconds
    epochs = mne.Epochs(raw, events, event_id, tmin, tmax,
                       proj=True, picks=picks,
                       baseline=None, preload=True)

    # Get data and labels
    X = epochs.get_data()
    y = epochs.events[:, -1]

    return X, y


In [41]:
def process_all_subjects(raw_dir, processed_dir, event_id=None, tmin=0, tmax=0.25,
                        l_freq=1.0, h_freq=40.0, montage='standard_1005'):
    """
    Process all EDF files for all subjects in the raw directory.

    Args:
        raw_dir (str): Path to directory containing subject folders with EDF files
        processed_dir (str): Path to directory to save processed data
        event_id (dict, optional): Mapping of event labels to integer codes
        tmin (float, optional): Start time before event (in seconds)
        tmax (float, optional): End time after event (in seconds)
        l_freq (float, optional): Low cutoff frequency for bandpass filter
        h_freq (float, optional): High cutoff frequency for bandpass filter
        montage (str, optional): Name of the electrode montage to use

    Returns:
        tuple: (X_all, y_all, subject_ids)
            X_all (np.ndarray): Combined feature array for all subjects
            y_all (np.ndarray): Combined label array for all subjects
            subject_ids (np.ndarray): Array indicating which subject each epoch belongs to
    """
    # Create processed directory if it doesn't exist
    Path(processed_dir).mkdir(parents=True, exist_ok=True)

    X_all = []
    y_all = []
    subject_ids = []  # To keep track of which subject each epoch belongs to

    # Get all subject directories
    subject_dirs = [d for d in Path(raw_dir).iterdir() if d.is_dir()]

    for subject_dir in sorted(subject_dirs):
        subject_id = subject_dir.name
        print(f"Processing subject {subject_id}...")

        # Process all EDF files for this subject
        for edf_file in subject_dir.glob('*.edf'):
            print(f"  Processing run: {edf_file.name}")

            try:
                # Process the file
                X, y = preprocess_edf(
                    str(edf_file),
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    l_freq=l_freq,
                    h_freq=h_freq,
                    montage=montage
                )

                X_all.append(X)
                y_all.append(y)
                # Add subject ID for each epoch
                subject_ids.extend([subject_id] * len(y))

            except Exception as e:
                print(f"Error processing {edf_file}: {str(e)}")
                continue

    # Combine all processed data
    if X_all:
        X_combined = np.concatenate(X_all, axis=0)
        y_combined = np.concatenate(y_all, axis=0)
        subject_ids = np.array(subject_ids)

        # Save processed data
        np.save(os.path.join(processed_dir, 'X_processed.npy'), X_combined)
        np.save(os.path.join(processed_dir, 'y_processed.npy'), y_combined)
        np.save(os.path.join(processed_dir, 'subject_ids.npy'), subject_ids)

        print(f"\nProcessing complete!")
        print(f"Total epochs: {len(y_combined)}")
        print(f"Data shape: {X_combined.shape}")
        print(f"Unique labels: {np.unique(y_combined)}")
        print(f"Number of subjects: {len(np.unique(subject_ids))}")

        return X_combined, y_combined, subject_ids
    else:
        print("No data was processed successfully!")
        return None, None, None

In [49]:
X_combined, y_combined, subject_ids = process_all_subjects(raw_dir,processed_dir)

Processing subject S001...
  Processing run: S001R11.edf
Extracting EDF parameters from /content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R11.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events fou

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished
<ipython-input-40-11b5cbb1b0f2>:26: RuntimeWarning: DigMontage is only a subset of info. There are 64 channel positions not present in the DigMontage. The channels missing from the montage are:

['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 41 original time points ...
0 bad epochs dropped
  Processing run: S001R08.edf
Extracting EDF parameters from /content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R08.edf...
EDF fi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished
<ipython-input-40-11b5cbb1b0f2>:26: RuntimeWarning: DigMontage is only a subset of info. There are 64 channel positions not present in the DigMontage. The channels missing from the montage are:

['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter

- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 41 original time points ...
0 bad epochs dropped
  Processing run: S001R07.edf
Extracting EDF parameters from /content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------


...

0 bad epochs dropped
  Processing run: S005R07.edf
Extracting EDF parameters from /content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S005/S005R07.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished
<ipython-input-40-11b5cbb1b0f2>:26: RuntimeWarning: DigMontage is only a subset of info. There are 64 channel positions not present in the DigMontage. The channels missing from the montage are:

['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 41 original time points ...
0 bad epochs dropped
  Processing run: S005R03.edf
Extracting EDF parameters from /content/motor-classification/data/MNE-eegbci-data/files/eegmmidb/1.0.0/S005/S005R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passb

[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished
<ipython-input-40-11b5cbb1b0f2>:26: RuntimeWarning: DigMontage is only a subset of info. There are 64 channel positions not present in the DigMontage. The channels missing from the montage are:

['Fc5.', 'Fc3.', 'Fc1.', 'Fcz.', 'Fc2.', 'Fc4.', 'Fc6.', 'C5..', 'C3..', 'C1..', 'Cz..', 'C2..', 'C4..', 'C6..', 'Cp5.', 'Cp3.', 'Cp1.', 'Cpz.', 'Cp2.', 'Cp4.', 'Cp6.', 'Fp1.', 'Fpz.', 'Fp2.', 'Af7.', 'Af3.', 'Afz.', 'Af4.', 'Af8.', 'F7..', 'F5..', 'F3..', 'F1..', 'Fz..', 'F2..', 'F4..', 'F6..', 'F8..', 'Ft7.', 'Ft8.', 'T7..', 'T8..', 'T9..', 'T10.', 'Tp7.', 'Tp8.', 'P7..', 'P5..', 'P3..', 'P1..', 'Pz..', 'P2..', 'P4..', 'P6..', 'P8..', 'Po7.', 'Po3.', 'Poz.', 'Po4.', 'Po8.', 'O1..', 'Oz..', 'O2..', 'Iz..'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your a


Processing complete!
Total epochs: 900
Data shape: (900, 64, 41)
Unique labels: [1 2 3]
Number of subjects: 5


Now we can split the training, validation, and testing sets.

In [43]:
def split_data(X, y, test_size=0.2, val_size=0.1, random_state=42):
    """
    Split data into train, validation, and test sets.

    Args:
        X (np.ndarray): Feature array.
        y (np.ndarray): Label array.
        test_size (float, optional): Fraction of data to use as test set. Defaults to 0.2.
        val_size (float, optional): Fraction of data to use as validation set. Defaults to 0.1.
        random_state (int, optional): Random seed. Defaults to 42.

    Returns:
        tuple: (X_train, X_val, X_test, y_train, y_val, y_test)
            X_train, X_val, X_test (np.ndarray): Feature splits.
            y_train, y_val, y_test (np.ndarray): Label splits.
    """
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=(test_size+val_size), random_state=random_state, stratify=y)
    val_relative = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=val_relative, random_state=random_state, stratify=y_temp)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [50]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_combined, y_combined)